### Random Forests and Photo-zs

In this notebook, we use Random Forests to estimate photometric redshifts starting from observations of galaxy magnitudes in six different photometric bands (u, g, r, i, z, y). It accompanies Chapter 6 of the book.

Essentially, we try to reproduce/improve upon the results of [this paper](https://arxiv.org/abs/1903.08174), for which the data are public and available [here](http://d-scholarship.pitt.edu/36064/).

Author: Viviana Acquaviva, with contributions by Jake Postiglione and Olga Privman.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)


font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
matplotlib.rcParams.update({'figure.autolayout': False})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [ ]:
import astropy

from astropy.io import fits

#fits stands for Flexible Image Transport System; it's a format that allows one to store images and summary data

Data import

It's easiest IMO to read the data into a data frame using pandas:

In [ ]:
with fits.open('../data/DEEP2_uniq_Terapix_Subaru_v1.fits') as data:
    df = pd.DataFrame(np.array(data[1].data).byteswap().newbyteorder()) #see https://numpy.org/devdocs/user/basics.byteswapping.html#changing-byte-ordering

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.shape

I can select the columns I want, corresponding to the brightness of the galaxies in the six bands of interest.

In [ ]:
features = df[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor']]

The target property is the redshift. For this catalog, spectroscopic (more precise) redshifts are available in the column below.

In [ ]:
target = df['zhelio']

In [ ]:
features.head(10)

In [ ]:
target.head(10)

### Ok, we are now ready to run our first Random Forest model!

To get an idea of what we are shooting for, we can look at the figure of the paper.

 ![Performance of photometric redshift reconstruction](Photoz_RF_CFHTLS_Deep.png)

In the figure above, $\sigma_{NMAD}$ is the normalized median absolute deviation of the residual vector, and $\eta$  is the fraction of outliers, defined as those objects for which (z_true - z_est)/(1+z_true) > 0.15.

To be fair, we are working with DEEP2/3 data, so our range is slightly different.

In [ ]:
model = RandomForestRegressor()

In [ ]:
model.get_params()

Establish benchmark.

In [ ]:
scores = cross_validate(model,features,target, cv = KFold(n_splits=5, shuffle=True, random_state=10), return_train_score=True)

In [ ]:
scores

Note - it takes a litle time! Also, the scores are the R2 score at this point.

In [ ]:
np.mean(scores['test_score'])

In [ ]:
np.mean(scores['train_score'])

### Learning Check-in
    
What issue do these scores indicate?

<br>

<details>
<summary style="display: list-item;">Click here for the answer!</summary>
<p>
    
```
It looks like we have a severe high variance issue.
```
    
</p>
</details>

Why?

<br>

<details>
<summary style="display: list-item;">Click here for the answer!</summary>
<p>

```
We can see a large discrepancy between the train and the test scores (to be fair, we should also look at the standard deviation of the train and test scores to confirm that the gap is statistically significant).
```
    
</p>
</details>
</br>


Let's also check the predictions:

In [ ]:
ypred = cross_val_predict(model,features,target, cv = KFold(n_splits=5, shuffle=True, random_state=10))

In [ ]:
plt.scatter(target,ypred, s = 20, c = 'royalblue')
plt.xlabel('True (spectroscopic) z', fontsize=14)
plt.ylabel('Predicted z',fontsize=14)
plt.axis('square')
plt.xlim(0,3)
plt.ylim(0,3)

### Question: Does it look like the one of the paper?

It's also interesting to look at the distribution of the predicted values, to see how they always tend to produce a narrower distribution. Why?

In [ ]:
plt.hist(target,bins=50,density=False,alpha=0.5, range = (0,3), label = 'True');
plt.hist(ypred,bins=50,density=False,alpha=0.5, range = (0,3), color = 'g', label = 'Predicted');
plt.legend(fontsize=14);

Ok, we are now ready to calculate the outlier fraction:

In [ ]:
len(np.where(np.abs(target-ypred)>0.15*(1+target))[0])/len(target)

And NMAD

In [ ]:
1.48*np.median(np.abs(target-ypred)/(1 + target)) 
#The 1.48 is there because for a Gaussian distribution, this becomes the standard deviation

### We have a pretty severe high variance issue, so we can perform some parameter optimization.

We can start by making the data set a bit smaller, as we have seen that timings were already challenging in simple k-fold CV.

In [ ]:
np.random.seed(20)
sel = np.random.choice(range(len(ypred)), 5000, replace = False) #sample without replacement

In [ ]:
len(np.unique(sel))

And we create our new smaller data set.

In [ ]:
seld = features.loc[sel,:]
selt = target[sel]

In [ ]:
littlescores = cross_validate(model,seld,selt, cv = KFold(n_splits=5, shuffle=True, random_state=10), return_train_score=True)

In [ ]:
littlescores['test_score'].mean(), littlescores['train_score'].mean()

Similar performance on training set ensures that size of data set is not a big issue and we can proceed with optimization.

#### Tree Parameters

The parameters associated to that are:

-  The minimum number of instances in a leaf node;

-  The minimum number of instances required in a split node;

- The maximum depth of tree.

-  The criterion chosen to decide whether a split is "worth it", expressed in terms of information gain;


#### Randomization Parameters

- The number of k < n features that are used in building trees.

- The re-sampling (boostrap) of the data set (T or F).


#### Forest Parameters

The number of trees in the forest (n_estimators) can be adjusted, with the general understanding that more trees are better, but at some point performance will plateau, so one can find the trade-off between having more trees and lower runtime.

In [ ]:
model.get_params()

Let's discuss a possible set.

- min_impurity_decrease 

- number of trees

- max_leaf_nodes

- min_samples_split

- max_features

In [ ]:
#Takes a few minutes

parameters = {'min_impurity_decrease':[0.1, 0.5, 0.0], \
              'max_features':[None,4,2], 'n_estimators':[50, 100, 200], 'min_samples_split': [10,20,100], 
              'max_leaf_nodes':[None, 100, 200]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(RandomForestRegressor(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(seld,selt)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)


In [ ]:
scoresCV

### And the verdict is....

We have NOT improved the test scores.




















### Time to look at data cleaning and/or imputing!

In my case, it was time to write to the authors of the paper, who told me exactly how they selected the data that went into making the learning set.

In [ ]:
mags = df[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor','subaru_source','cfhtls_source','zquality']]

In [ ]:
mags.head()

In [ ]:
mags.shape

In [ ]:
#redshift quality - only use objects with high-quality spectroscopic redshift measurements

mags = mags[mags['zquality'] >= 3]

mags.shape

In [ ]:
#cfhtls deep photo 

mags = mags[mags['cfhtls_source'] == 0]

mags.shape

In [ ]:
#Let's leave this out for now (subaru deep photo)

#mags = mags[mags['subaru_source'] == 0]

#mags.shape

Unavailable measurements are marked by -99 or 99 (while typical values are around 20-25). We can get rid of data with missing measurements.

In [ ]:
mags = mags[mags > -10].dropna()

In [ ]:
mags.shape

In [ ]:
mags = mags[mags < 90].dropna()

In [ ]:
mags.shape

Our final set is made of the six original features and it has 6,307 objects.

In [ ]:
sel_features = mags[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor']]
sel_features.head()

We need, of course, to select the same set on the target vector.

In [ ]:
sel_target = target[sel_features.index]

Let's see how our benchmark model works. Note that for reproducible results we need to fix the random\_state parameter of the Random Forest (which controls the bootstrap process) and the random seed of the cross validation. 

In [ ]:
scores = cross_validate(RandomForestRegressor(random_state = 5),sel_features,sel_target,cv = KFold(n_splits=5, shuffle=True, random_state=10), \
               return_train_score=True)

In [ ]:
print(np.round(np.mean(scores['test_score']),3), np.round(np.std(scores['test_score']),3))

In [ ]:
print(np.round(np.mean(scores['train_score']),3), np.round(np.std(scores['train_score']),3))

The scores have improved noticeably! However, we can still observe high variance. We can re-run the optimization process (note that the data set size is limited, so we don't need to make it smaller).

In [ ]:
#This now takes ~3 minutes

parameters = {'max_depth':[3, 6, None], \
              'max_features':[None,4,2], 'n_estimators':[50,100,200], 'min_samples_leaf': [1,5,10]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(RandomForestRegressor(random_state = 5), parameters, cv = KFold(n_splits=5, shuffle=True, random_state=10), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(sel_features,sel_target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV

### Learning Check-in
    
Based on the results from the Grid Search, would you expect that a significant improvement come come from enlarging the space of parameters?

<br>
<details><summary><b>Click here for the answer!</b></summary>
<p>
    
```
Probably not, because the scores are flat-ish over the first 10-20 models, suggesting that further optimization is unlikely to help significantly.
```
    
</p>
</details>

In [ ]:
bm = model.best_estimator_

We can generate one set of predictions to visualize what happens.

In [ ]:
ypred = cross_val_predict(bm, sel_features,sel_target, cv = KFold(n_splits=5, shuffle=True, random_state=10))

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(sel_target,ypred, s =10)
plt.xlabel('z_spec')
plt.ylabel('z_photo')
plt.ylim(0,2)
plt.xlim(0,2)

Calculate outlier fraction and compare with the figure.

In [ ]:
len(np.where(np.abs(sel_target-ypred)>0.15*(1+sel_target))[0])/len(sel_target)

Calculate Normalized Median Absolute Deviation (NMAD).

In [ ]:
1.48*np.median(np.abs(sel_target-ypred)/(1 + sel_target))

### Conclusion: How does our model compare with the paper's results?

We are doing slightly worse, but we have a secret weapon in feature engineering.


### Feature engineering exercise: what happens if I use colors instead of magnitudes? (below!)

In [ ]:
sel_features.loc[:,'u-g'] = sel_features['u_apercor'] - sel_features['g_apercor']
sel_features.loc[:,'g-r'] = sel_features['g_apercor'] - sel_features['r_apercor']
sel_features.loc[:,'r-i'] = sel_features['r_apercor'] - sel_features['i_apercor']
sel_features.loc[:,'i-z'] = sel_features['i_apercor'] - sel_features['z_apercor']
sel_features.loc[:,'z-y'] = sel_features['z_apercor'] - sel_features['y_apercor']

In [ ]:
sel_colors = sel_features[['u-g','g-r','r-i','i-z','z-y','i_apercor']]

In [ ]:
scores = cross_validate(RandomForestRegressor(),sel_colors,sel_target,cv = KFold(n_splits=5, shuffle=True, random_state=10), \
               return_train_score=True)

In [ ]:
scores 

In [ ]:
scores['test_score'].mean(), scores['test_score'].std()

In [ ]:
parameters = {'max_depth':[3, 6, None], \
              'max_features':[None,4,2], 'n_estimators':[50,100,200], 'min_samples_leaf': [1,5,10]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(RandomForestRegressor(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(sel_colors, sel_target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score','mean_fit_time']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV

In [ ]:
bm

In [ ]:
bm = model.best_estimator_

In [ ]:
ypred = cross_val_predict(bm, sel_colors, sel_target, cv = KFold(n_splits=5, shuffle=True, random_state=10))

### Learning Check-in
    
Calculate the Normalized Median Absolute Deviation (NMAD) and the outlier fraction of the predicted redshifts with respect to the true redshifts by filling in the code below.

```python
1.48 * np.median(... (... - ...)/(1 + ...))

len(... (np.abs(...) > ... * (1 + ...))[0]) / len(...)
```

<br>

<details>
<summary style="display: list-item;">Click here for the answer!</summary>
<p>
    
```python
1.48 * np.median(np.abs(sel_target-ypred)/(1 + sel_target))

len(np.where(np.abs(sel_target-ypred)>0.15*(1+sel_target))[0])/len(sel_target)
```
    
</p>
</details>
</br>

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(sel_target,ypred, s =10)
plt.ylim(0,2)
plt.xlim(0,2)

We finally matched the performance of the paper (although, to be fair, we are not using *exactly* the same set of data).

### <font color='blue'> Until now, we have never discussed a very important point: how to estimate the uncertainty associated to our results. </font>

One source of scatter on the global performance metrics comes from the system's architecture: we should generate a bunch of predictions with several random seeds. This is equivalent to the scatter found in the monitored metric (e.g., MSE or r2 score) in cross validation.

In [ ]:
model = RandomForestRegressor(max_features=4, n_estimators=200) #I need to re-seed the random state

In [ ]:
#Note: this also takes time!

seeds = np.random.choice(100,8, replace = False) #pick 8

olf = np.zeros(8)
NMAD = np.zeros(8)

for i in range(8): #A bit rough, but it gives a sense of what happens by varying the random seeds!
    print('Iteration', i)
    ypred = cross_val_predict(RandomForestRegressor(max_features=4, n_estimators=200,random_state=seeds[i]), sel_features, sel_target, cv = KFold(n_splits=5, shuffle=True, random_state=seeds[i]))
    olf[i] = len(np.where(np.abs(sel_target-ypred)>0.15*(1+sel_target))[0])/len(sel_target)
    NMAD[i] = 1.48*np.median(np.abs(sel_target-ypred)/(1 + sel_target))

print('OLF avg/std:, {0:.5f}, {1:0.5f}'.format(olf.mean(), olf.std()))
print('NMAD avg/std:, {0:.5f}, {1:0.5f}'.format(NMAD.mean(), NMAD.std()))

#### However, we should also think about how to quantify the observational error on our *individual* inputs.

The literature on this subjects is scarce, but my proposal is to run a forward-pass of the best model a bunch of times, with different inputs that are drawn from modeling each input according to their noise profile (e.g. a Gaussian where mean = observed value and sigma = experimental error).

Assuming that the noise profile doesn't change from the "true" value to the "observed" value, this should include the experimental error and the "limited information" error, which comes from limited training set size, lack of informative features, model architecture etc. This has been described as epistemic vs aleatoric uncertainty (see [this recent review](https://link.springer.com/article/10.1007/s10994-021-05946-3)).